In [4]:
import numpy as np
from Capa import Capa

---

In [5]:
class Red_Neuronal():
    def __init__(self,num_neuronas_capa, f_act_capa, num_inputs_1):
        self.lista_capas = []
        self.lista_capas.append(Capa(num_neuronas_capa[0], num_inputs_1, f_act_capa[0]))
        for i in range(1, len(num_neuronas_capa)):
            c = Capa(num_neuronas_capa[i], num_neuronas_capa[i-1], f_act_capa[i])
            self.lista_capas.append(c)

    # Función para hacer los cálculos
    def calc(self, inputs):
        for c in self.lista_capas:
            inputs = c.get_output(inputs)
        return inputs

    # Función para entrenar
    def train(self):
        return None


    # Función para mostrar los datos de la red neuronal
    def print(self):
        c = 1
        for capa in self.lista_capas:
            print("Capa c: ")
            capa.print()
            print("---------")
            c += 1

In [6]:
rn1 = Red_Neuronal([3,2,1], [4,2,1], 4)
rn1.print()
print(rn1.calc([[4, -2, 3, -5]]))

Capa c: 
Inputs: 
 [[0.57181287 0.10874181 0.26991485]
 [0.0578009  0.66404357 0.38903514]
 [0.51494712 0.38474887 0.45648311]
 [0.75044035 0.00278653 0.15171477]
 [0.72423653 0.61750481 0.18055057]]
Input shape: 
 (5, 3)
F Act: 
 4
---------
Capa c: 
Inputs: 
 [[0.49272948 0.99038423]
 [0.32324813 0.46934634]
 [0.53396289 0.98604604]
 [0.43727786 0.1571828 ]]
Input shape: 
 (4, 2)
F Act: 
 2
---------
Capa c: 
Inputs: 
 [[0.55993021]
 [0.41475758]
 [0.18515671]]
Input shape: 
 (3, 1)
F Act: 
 1
---------
[1.]
